<a href="https://colab.research.google.com/github/DAS9051/Portfolio-Optimization-with-Mean-Variance-Analysis/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

Fetch Stock Download Data

In [ ]:
stocks = [
    "AAPL", "GOOGL", "MSFT", "TSLA", "AMZN",  # Tech
    "JPM", "BAC", "GS",  # Banking
    "XOM", "CVX",  # Energy
    "JNJ", "PFE",  # Healthcare
    "WMT", "TGT",  # Retail
    "NVDA", "AMD",  # Semiconductors
    "DIS", "NFLX",   # Entertainment
    "ACN","ADBE","GOOG"
]



data = yf.download(stocks, start="2020-01-01", end="2025-03-01")
data = data["Close"]
print(data.head())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  21 of 21 completed


Ticker           AAPL         ACN        ADBE        AMD       AMZN  \
Date                                                                  
2020-01-02  72.716072  195.263565  334.429993  49.099998  94.900497   
2020-01-03  72.009109  194.938354  331.809998  48.599998  93.748497   
2020-01-06  72.582893  193.665436  333.709991  48.389999  95.143997   
2020-01-07  72.241562  189.484192  333.390015  48.250000  95.343002   
2020-01-08  73.403641  189.855865  337.869995  47.830002  94.598503   

Ticker            BAC        CVX         DIS       GOOG      GOOGL  ...  \
Date                                                                ...   
2020-01-02  31.275541  96.158691  146.391541  68.123726  68.186821  ...   
2020-01-03  30.626163  95.826118  144.712265  67.789429  67.830109  ...   
2020-01-06  30.582285  95.501427  143.872635  69.460922  69.638054  ...   
2020-01-07  30.380447  94.281929  143.922028  69.417572  69.503548  ...   
2020-01-08  30.687586  93.204971  143.625687  69.964

In [ ]:
fundamentals = {}

for stock in stocks:
    ticker = yf.Ticker(stock)

    # Fetch fundamental metrics
    pe_ratio = ticker.info.get("trailingPE", None)  # P/E Ratio

    # Use Operating Income
    operating_income = ticker.financials.loc["Operating Income"].sum() if "Operating Income" in ticker.financials.index else None

    # Use Research and Development (R&D)
    r_and_d = ticker.financials.loc["Research And Development"].sum() if "Research And Development" in ticker.financials.index else None

    profit_margin = ticker.info.get("profitMargins", None)  # Profit Margin


    fundamentals[stock] = {
        "P/E": pe_ratio,
        "Operating Income": operating_income,
        "R&D": r_and_d,
        "Profit Margin": profit_margin
    }

fundamentals_df = pd.DataFrame(fundamentals).T
print(fundamentals_df)

              P/E  Operating Income           R&D  Profit Margin
AAPL    37.887480      4.659030e+11  1.094500e+11        0.24295
GOOGL   21.651308      3.502390e+11  1.658150e+11        0.28604
MSFT    31.641994      3.512550e+11  1.019330e+11        0.35428
TSLA   128.131710      3.697900e+10  1.417700e+10        0.07259
AMZN    35.965706      1.425720e+11  4.274000e+10        0.09287
JPM     12.267342               NaN           NaN        0.35060
BAC     12.897197               NaN           NaN        0.28243
GS      13.812191               NaN           NaN        0.27368
XOM     13.905612      1.721600e+11           NaN        0.09889
CVX     16.084362      1.012760e+11           NaN        0.09031
JNJ     28.839100      8.751400e+10  6.072900e+10        0.15836
PFE     18.957447      8.117500e+10  4.328900e+10        0.12622
WMT     38.058090      9.593000e+10           NaN        0.02854
TGT     12.988714      2.504000e+10           NaN        0.03839
NVDA    38.329933      1.

Count number of Stocks in the portfolio

In [ ]:
num_stocks = len(data.columns)
num_stocks

21

Setup some Constants

In [ ]:
weights = np.ones(num_stocks) / num_stocks
returns = data.pct_change().dropna()
mean_returns = returns.mean() * 252


Makes the Covariance Matirx

In [ ]:
# Fill missing values with median to prevent errors
fundamentals_df.fillna(fundamentals_df.median(), inplace=True)

# Normalize the fundamental metrics (scale between 0 and 1)
scaler = MinMaxScaler()
normalized_fundamentals = scaler.fit_transform(fundamentals_df)

# Define fundamental-based risk model
# Higher P/E increases risk, higher Operating Income and Profit Margins lower risk
fundamental_risk = (1 / (normalized_fundamentals[:, 0] + 0.01)) - (normalized_fundamentals[:, 1]) - (normalized_fundamentals[:, 3]) + (normalized_fundamentals[:, 2] * 0.5)

# Convert to a diagonal covariance matrix
fundamental_cov_matrix = np.diag(fundamental_risk)

Mean Variance Analysis

In [ ]:
def portfolio_performance(weights, mean_returns, cov_matrix):
    portfolio_return = np.dot(weights, mean_returns)  # Expected return
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))  # Risk (volatility)
    return portfolio_return, portfolio_risk

def minimize_volatility(weights, mean_returns, cov_matrix):
    return portfolio_performance(weights, mean_returns, cov_matrix)[1]

In [ ]:
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

# Bounds: No short-selling (weights must be between 0 and 1)
bounds = [(0, 1) for _ in range(num_stocks)]

# Optimization process
optimized = minimize(minimize_volatility, weights, args=(mean_returns, fundamental_cov_matrix),
                     method='SLSQP', bounds=bounds, constraints=constraints)

# Extract optimal portfolio weights
optimal_weights = optimized.x

Returns Portfolio Optimization Precents

In [ ]:
for stock, weight in zip(stocks, optimal_weights):
    print(f"{stock}: {weight:.2%}")

AAPL: 6.47%
GOOGL: 2.04%
MSFT: 4.59%
TSLA: 24.55%
AMZN: 4.82%
JPM: 0.21%
BAC: 0.33%
GS: 0.50%
XOM: 0.51%
CVX: 0.91%
JNJ: 3.34%
PFE: 1.44%
WMT: 4.98%
TGT: 0.34%
NVDA: 6.84%
AMD: 16.59%
DIS: 4.19%
NFLX: 6.94%
ACN: 3.23%
ADBE: 5.09%
GOOG: 2.09%
